In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model, Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
#from keras.applications.imagenet_utils import preprocess_input
#import pydot
#from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
#import scipy.misc
import matplotlib.pyplot as plt
from keras.optimizers import Adam
%matplotlib inline

Using TensorFlow backend.


In [3]:
def identity_block(X, f, filters):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shape = Input((200,200,3))
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization()(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [4]:
def convolutional_block(X, f, filters, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shape = Input((200,200,3))
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), padding="same", kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization()(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'same',kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization()(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [5]:
def generator_model_ResNet50(input_shape=(200, 200, 3)):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL(X) -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL(X) -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(1, 1), kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    # X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], s = 2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    # X = AveragePooling2D((2,2), name="avg_pool")(X)

    ### END CODE HERE ###
    # output layer
    X = Conv2D(3, (3, 3), padding='same', kernel_initializer=glorot_uniform(seed=0))(X) # 여기서 Conv2D 의 the amount of filters 는 채널갯수!! RGB는 3개 , GRAY는 1개, RGBA는 4개 겠지.
    X = BatchNormalization()(X)
    gen_img = Activation('sigmoid')(X)
    
    # Create model
    model = Model(inputs=X_input, outputs=gen_img, name='Generator_Model_ResNet50')
    return model

In [6]:
model = generator_model_ResNet50()

W1104 14:05:53.297782 139652093036352 deprecation_wrapper.py:119] From /home/zoshs2/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1104 14:05:53.328556 139652093036352 deprecation_wrapper.py:119] From /home/zoshs2/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1104 14:05:53.335902 139652093036352 deprecation_wrapper.py:119] From /home/zoshs2/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1104 14:05:53.373037 139652093036352 deprecation_wrapper.py:119] From /home/zoshs2/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is de

In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 206, 206, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 200, 200, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 200, 200, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation

In [5]:
def discriminator_model(input_shape=(200, 200, 3)):
    
    X_input = Input(input_shape)
    
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='valid', input_shape=(200,200,3), kernel_initializer=glorot_uniform(seed=0)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(32, (3,3), padding='valid', kernel_initializer=glorot_uniform(seed=0)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(64, (3,3), padding='valid', kernel_initializer=glorot_uniform(seed=0)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    
    ## FC
    model.add(Dense(64, kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(32, kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    valid = model.add(Dense(1, activation='sigmoid'))
    
    model = Model(inputs=model, outputs=valid, name='Discriminator')
    
    return model

In [6]:
def generator_containing_discriminator(generator, discriminator):
    inputs = Input(shape=(200,200,3))
    discriminator.trainable = False
    generated_image = generator(inputs)
    outputs = discriminator(generated_image)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [7]:
g = generator_model_ResNet50()

W1101 17:24:09.037894 140013654660928 deprecation_wrapper.py:119] From /home/zoshs2/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1101 17:24:09.059645 140013654660928 deprecation_wrapper.py:119] From /home/zoshs2/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1101 17:24:09.067770 140013654660928 deprecation_wrapper.py:119] From /home/zoshs2/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1101 17:24:09.114616 140013654660928 deprecation_wrapper.py:119] From /home/zoshs2/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is de

In [ ]:
def blurred_and_clean_preprocess(blurred_data, clean_data, seed=False):
    if seed=True:
        pass
    
    

In [ ]:
def Train_GAN(blurred_img, clean_img, epochs=200, batch_size=128, save_interval=50, seed=0):
    np.random.seed(seed)
    np.random.shuffle(blurred_img)
    np.random.seed(seed)
    np.random.shuffle(clean_img)
    half_batch = int(batch_size / 2)
    opt = Adam(0.0002, 0.5)
    optimizer=opt
    
    d = discriminator_model()
    d.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    g = generator_model_ResNet50()
    g.compile(loss='binary_crossentropy', optimizer=opt)
    for epoch in range(epochs):
        ######################
        # Train Discriminator
        ######################
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_imgs = Y_train[idx]
        blur_imgs = X_train[idx]
        
        gen_imgs = g.predict(blur_img)

In [16]:
class GAN():
    def __init__(self):
        self.img_rows = 200
        self.img_cols = 200
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.discriminator_model()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.generator_model_ResNet50()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(self.img_shape)
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)
        
        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    def generator_model_ResNet50(self):
        """
        Implementation of the popular ResNet50 the following architecture:
        CONV2D -> BATCHNORM -> RELU -> MAXPOOL(X) -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
        -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL(X) -> TOPLAYER

        Arguments:
        input_shape -- shape of the images of the dataset
        classes -- integer, number of classes

        Returns:
        model -- a Model() instance in Keras
        """

        # Define the input as a tensor with shape input_shape
        X_input = Input(self.img_shape)

        # Zero-Padding
        X = ZeroPadding2D((3, 3))(X_input)

        # Stage 1
        X = Conv2D(64, (7, 7), strides=(1, 1), kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization()(X)
        X = Activation('relu')(X)
        
        # X = MaxPooling2D((3, 3), strides=(2, 2))(X)

        # Stage 2
        X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
        X = identity_block(X, 3, [64, 64, 256])
        X = identity_block(X, 3, [64, 64, 256])
        
        ### START CODE HERE ###

        # Stage 3 (≈4 lines)
        X = convolutional_block(X, f = 3, filters = [128, 128, 512], s = 2)
        X = identity_block(X, 3, [128, 128, 512])
        X = identity_block(X, 3, [128, 128, 512])
        X = identity_block(X, 3, [128, 128, 512])
        
        # Stage 4 (≈6 lines)
        X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
        X = identity_block(X, 3, [256, 256, 1024])
        X = identity_block(X, 3, [256, 256, 1024])
        X = identity_block(X, 3, [256, 256, 1024])
        X = identity_block(X, 3, [256, 256, 1024])
        X = identity_block(X, 3, [256, 256, 1024])
        
        # Stage 5 (≈3 lines)
        X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
        X = identity_block(X, 3, [512, 512, 2048])
        X = identity_block(X, 3, [512, 512, 2048])
        
        # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
        # X = AveragePooling2D((2,2), name="avg_pool")(X)

        ### END CODE HERE ###
        # output layer
        X = Conv2D(3, (3, 3), padding='same', kernel_initializer=glorot_uniform(seed=0))(X) # 여기서 Conv2D 의 the amount of filters 는 채널갯수!! RGB는 3개 , GRAY는 1개, RGBA는 4개 겠지.
        X = BatchNormalization()(X)
        gen_img = Activation('sigmoid')(X)
        
        # Create model
        model = Model(inputs=X_input, outputs=gen_img, name='Generator_Model_ResNet50')
        return model
    
    def discriminator_model(self):
        
        X_input = Input(shape=self.img_shape)
        
        x = Conv2D(32, (3,3), padding='valid', input_shape=(self.img_shape), kernel_initializer=glorot_uniform(seed=0))(X_input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=2)(x)

        x = Conv2D(32, (3,3), padding='valid', kernel_initializer=glorot_uniform(seed=0))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=2)(x)

        x = Conv2D(64, (3,3), padding='valid', kernel_initializer=glorot_uniform(seed=0))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D(pool_size=(2,2), strides=2)(x)
        x = Flatten()(x)

        ## FC
        x = Dense(64, kernel_initializer='he_uniform')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dense(32, kernel_initializer='he_uniform')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dense(1, activation='sigmoid')(x)

        model = Model(inputs=X_input, outputs=x, name='Discriminator')

        return model
    
    def train(self, blur_arr, clean_arr, epochs=100, batch_size=128, save_interval=50):
        # Load the dataset
        X_train = blur_arr
        Y_train = clean_arr
        half_batch = int(batch_size/2)
        
        for epoch in range(epochs):
            ######################
            # Train Discriminator
            ######################
            
            idx = np.random.randint(0, Y_train.shape[0], half_batch)
            clean_imgs = Y_train[idx]
            blurred_imgs = X_train[idx]
            gen_imgs = self.generator.predict(blurred_imgs)
            
            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(clean_imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            ######################
            # Train Generator
            ######################
            valid_y = np.array([1] * batch_size)
            
            # Train the generator
            g_loss = self.combined.train_on_batch(blurred_imgs, valid_y)
            
            # Print the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            

In [17]:
gan = GAN()

In [8]:
blurred_img = np.load("/home/zoshs2/tf_gpu/Re/DATA/BS2_BW1_NN_RGB_10318.npy")
clean_img = np.load("/home/zoshs2/tf_gpu/Re/DATA/BS0_BW0_NN_RGB_10318.npy")

In [9]:
blurred_img = blurred_img['Img']
clean_img = clean_img['Img']

In [21]:
blurred_img.shape

(10318, 200, 200, 3)

In [23]:
clean_img.shape

(10318, 200, 200, 3)

In [29]:
idx = np.random.randint(0, blurred_img.shape[0], 32)

In [31]:
blurred_img[idx].shape

(32, 200, 200, 3)

In [18]:
blurred_img_nor = blurred_img / 255.
clean_img_nor = clean_img / 255.

In [ ]:
gan.train(blurred_img_nor, clean_img_nor, batch_size=32, save_interval=50)